#### 환경 변수 설정

In [1]:
import os 

os.environ["LANGCHAIN_PROJECT"] = "GEMINI_PROJECT"

In [2]:
# 토큰 정보 불러오기 
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
gemini_api_key = os.getenv("Gemini_API_KEY")

In [4]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

d:\envs\want2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
api_key = os.getenv("Gemini_API_KEY")
if api_key is None:
    raise ValueError("Gemini_API_KEY 환경 변수가 설정되지 않았습니다.")

#### 모델 사용해보기 - 기본

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key
)

model.invoke("달력이 영어로 뭐야")

AIMessage(content='"Calendar"는 달력을 의미하는 영어 단어입니다. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-17338876-5bf6-49a6-8eb7-a02a1fe1f453-0', usage_metadata={'input_tokens': 10, 'output_tokens': 15, 'total_tokens': 25})

#### 모델 사용해보기 - 스트리밍

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=api_key
)

response = model.stream("달력이 영어로 뭐야")

for token in response:
    print(token.content, end="", flush=True)

영어로 달력은 **calendar** 입니다. 


#### 모델 사용해보기 - Chat

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key
)

model.invoke(
    [
        SystemMessage(
            content = "1. O, X로 답한 후, 2. X라면 올바른 답을 알려주세요"
        ),
        HumanMessage(
            content = "사과는 영어로 'Orange'입니다"
        ),
    ]
).content

'X\n\n사과는 영어로 "Apple"입니다. \n'

#### Chain 사용하기 

In [9]:
from pathlib import Path
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

In [10]:
def read_oldman(x):
    oldman_path = Path("../persona/oldman.txt")
    return oldman_path.read_text(encoding="utf-8")

In [32]:
def read_prompt(file_path):
    with open(file_path, 'r') as file:
        return file.read()

template = read_prompt("D:/envs/AI/static/templates/Demo.prompt")

In [33]:
# 세션 상태 초기화
session_state = {}

# 키 값들 정의
key_chain = "chat_chain"
key_memory = "chat_memory"
key_history = "chat_history"

# 채팅을 이어나갈 때
if key_chain in session_state:
    chain = session_state[key_chain]
    memory = session_state[key_memory]
    history = session_state[key_history]
# 새로운 템플릿을 적용할 때
else:
    # 초기화 
    session_state[key_history] = []

    # 메모리 설정
    memory = ConversationBufferMemory(
        return_messages=True, 
        memory_key="chat_history"
    )
    # 프롬프트 설정
    def read_prompt(file_path):
        with open(file_path, 'r') as file:
            return file.read()
    
    template = read_prompt("D:/envs/AI/static/templates/Demo.prompt")
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    # 체인 만들기
    runnable = RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables)
        | itemgetter("chat_history")
    )
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key)
    output_parser = StrOutputParser()
    runnable = RunnablePassthrough.assign(
            chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
        )
    chain = runnable | prompt | model | output_parser
    # 세션 정보 저장
    session_state[key_chain] = chain
    session_state[key_memory] = memory


In [40]:
def chat():
    # AI 캐릭터의 첫 인사말
    greeting = get_greeting()
    print(f"AI: {greeting}")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("AI: 대화를 종료합니다. 좋은 하루 되세요!")
            break

        # 사용자 입력을 처리하고 AI 응답 생성
        response = chain.invoke({"input": user_input})
        print(f"AI: {response}")

In [38]:
def chat():
    # AI 캐릭터의 첫 인사말
    greeting = get_greeting()
    print(f"AI: {greeting}")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("AI: 대화를 종료합니다. 좋은 하루 되세요!")
            break

        # 사용자 입력을 처리하고 AI 응답 생성
        response = chain.invoke({"input": user_input})
        print(f"AI: {response}")

        # 메모리에 대화 저장
        memory.save_context({"input": user_input}, {"output": response})

In [41]:
# 대화 시작
print("대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 'bye'를 입력하세요.")
chat()


대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 'bye'를 입력하세요.


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\envs\\AI\\persona\\oldman.txt'

In [36]:
# 대화 시작
if __name__ == "__main__":
    print("대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 'bye'를 입력하세요.")
    chat()

대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 'bye'를 입력하세요.


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\envs\\AI\\persona\\oldman.txt'

In [31]:
import os 
from pathlib import Path
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

load_dotenv()

api_key = os.getenv("Gemini_API_KEY")
if api_key is None:
    raise ValueError("Gemini_API_KEY 환경 변수가 설정되지 않았습니다.")

def read_oldman():
    oldman_path = Path("D:/envs/AI/persona/oldman.txt")
    with oldman_path.open(encoding="utf-8") as file:
        return file.readlines()

def get_greeting():
    lines = read_oldman()
    for line in lines:
        if line.startswith("greeting ="):
            return line.split('=')[1].strip()

template = read_prompt("D:/envs/AI/static/templates/Demo.prompt")

# 세션 상태 초기화
session_state = {}

# 키 값들 정의
key_chain = "chat_chain"
key_memory = "chat_memory"
key_history = "chat_history"

if key_chain in session_state:
    chain = session_state[key_chain]
    memory = session_state[key_memory]
    history = session_state[key_history]
else:
    session_state[key_history] = []

    memory = ConversationBufferMemory(
        return_messages=True, 
        memory_key="chat_history"
    )
    
    template = read_prompt("D:/envs/AI/static/templates/Demo.prompt")
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )

    runnable = RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    )
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key)
    output_parser = StrOutputParser()
    chain = runnable | prompt | model | output_parser

    session_state[key_chain] = chain
    session_state[key_memory] = memory

def chat():
    greeting = get_greeting()
    print(f"AI: {greeting}")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("AI: 대화를 종료합니다. 좋은 하루 되세요!")
            break

        response = chain.invoke({"input": user_input})
        print(f"AI: {response}")

        memory.save_context({"input": user_input}, {"output": response})

if __name__ == "__main__":
    print("대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 'bye'를 입력하세요.")
    chat()


대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 'bye'를 입력하세요.


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\envs\\AI\\persona\\oldman.txt'